# **Multi-Agent Chatbot LangGraph Workflow**

### **Overview**
- **Orchestrator Agent:** Routes user queries, aggregates responses, and finalizes replies.  
- **SQL Agent:** Handles database-related queries and retrieves structured data.  
- **Enterprise Search Agent:** Handls document-related queries and retrieves unstructured text data.  
- **Reporting Agent:** Generates a structured report when explicitly requested by the user.  
- **Chat History:** Tracks all interactions, providing context across multiple queries.

## **1. State Definition**

Tracks the data moving between agents. Maintains context across the workflow.

In [ ]:
class State:
    def __init__(self, user_query, chat_history=None, request_report=False):
        self.user_query = user_query  # Current query from the user
        self.route = None             # 'sql' or 'search'
        self.sql_result = None        # Result from SQL Agent
        self.search_result = None     # Result from Search Agent
        self.final_response = None    # Aggregated final response
        self.report = None            # Structured report from Reporting Agent
        self.request_report = request_report  # True if user requested a report
        self.chat_history = chat_history or []  # Stores previous interactions

## **2. Agents**

### **Orchestrator Agent**

- Handles routing based on query type.  
- Aggregates results after SQL or Search Agent completes their tasks.  
- Finalizes response or invokes the Reporting Agent if a report is requested.

In [ ]:
def orchestrator_agent(state: State):
    print("\nChat History:")
    for entry in state.chat_history:
        print(entry)

    if state.route is None:
        if "database" in state.user_query or "SQL" in state.user_query:
            state.route = "sql"
            return "sql_agent"
        else:
            state.route = "search"
            return "search_agent"

    if state.route == "search":
        state.final_response = f"Search Result: {state.search_result}"

    elif state.route == "sql":
        state.final_response = f"Database Result: {state.sql_result}"
    state.chat_history.append({
        "user_query": state.user_query,
        "agent": state.route,
        "response": state.final_response
    })

    return "reporting_agent" if state.request_report else state

### **SQL Agent**

- Processes SQL-related queries.  
- Uses chat history for context if available.  
- Returns SQL query results to the Orchestrator Agent.

In [ ]:
def sql_agent(state: State):
    context = "\n".join([f"User: {entry['user_query']}, Agent: {entry['agent']}, Response: {entry['response']}" 
                         for entry in state.chat_history])
    state.sql_result = f"SQL Query Result for '{state.user_query}' with context:\n{context}"
    return "orchestrator_agent"

### **Enterprise Search Agent**

- Processes document-related queries.  
- Uses chat history for context if available.  
- Returns search results to the Orchestrator Agent.

In [ ]:
def search_agent(state: State):
    context = "\n".join([f"User: {entry['user_query']}, Agent: {entry['agent']}, Response: {entry['response']}" 
                         for entry in state.chat_history])
    state.search_result = f"Search Result for '{state.user_query}' with context:\n{context}"
    return "orchestrator_agent"

### **Reporting Agent**

- Uses the final aggregated response from the Orchestrator Agent.  
- Creates a structured report from the aggregated data.

In [ ]:
def reporting_agent(state: State):
    state.report = f"Aggregated Report:\n{state.final_response}\nGenerated using results from your conversation."
    state.chat_history.append({
        "user_query": state.user_query,
        "agent": "reporting",
        "response": state.report
    })
    state.final_response = state.report
    return state